<a href="https://colab.research.google.com/github/samanthajmichael/dsba6211-summer2024/blob/main/notebooks/dsba6211_summer2024_lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Started with Prompt Engineering

This notebook contains examples and exercises to learning about prompt engineering. It was originally created by DAIR.AI | Elvis Saravia with modifications.

We will be using the [OpenAI APIs](https://platform.openai.com/) for all examples. I am using the default settings `temperature=0.7` and `top-p=1`

---

## 1. Prompt Engineering Basics: OpenAI API and configurations


Below we are loading the necessary libraries, utilities, and configurations.

In [1]:
%%capture
# update or install the necessary libraries
!pip install --upgrade openai==1.35.1
!pip install --upgrade langchain==0.2.5
!pip install --upgrade langchain-openai==0.1.8
!pip install langchain-community==0.2.5

In [2]:
import openai
import os
import IPython
from langchain.llms import OpenAI
from google.colab import userdata

Load environment variables. Since I'm running this in a Colab notebook, I'm using `userdata.get()`. Just make sure to add your API Key into your Colab.

![](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*5wEevNCOf80GTHwptPTB4g.png)

Alternatively, you can use `python-dotenv` with a `.env` file with your `OPENAI_API_KEY` then load it.

In [3]:
# API configuration
openai.api_key = userdata.get("OPENAI_API_KEY")

# for LangChain
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

### Types of Endpoints

OpenAI (and other LLM) API's typically have two types API endpoints: completion and chat.

![](https://substackcdn.com/image/fetch/w_1456,c_limit,f_webp,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2Fccf3b37b-ad35-43b3-8cac-607a01473ba8_2719x508.png)

Source: [Generally Intelligent SubStack](https://generallyintelligent.substack.com/p/chat-vs-completion-endpoints)

Originally, the completions endpoint was the first endpoint. However, OpenAI has [announced](https://community.openai.com/t/completion-models-are-now-considered-legacy/656302) it is deprecating that endpoint.

`/completions` endpoint provides the completion for a single prompt and takes a single string as an input, whereas the `/chat/completions` provides the responses for a given dialog and requires the input in a specific format corresponding to the message history. Instead of taking a `prompt`, Chat models take a list of `messages` as input and return a model-generated message as output.



| MODEL FAMILIES               | EXAMPLES                                         | API ENDPOINT                               |
|------------------------------|--------------------------------------------------|--------------------------------------------|
| Newer models (2023–)         | gpt-4, gpt-4-turbo-preview, gpt-3.5-turbo        | https://api.openai.com/v1/chat/completions |
| Updated legacy models (2023) | gpt-3.5-turbo-instruct, babbage-002, davinci-002 | https://api.openai.com/v1/completions      |


Although the chat format is designed to make multi-turn conversations easy, it’s just as useful for single-turn tasks without any conversation.

We're going to the use the Chat `openai.chat.completions` endpoint.

For more details, check out [OpenAI's docs](https://platform.openai.com/docs/guides/text-generation/chat-completions-vs-completions).

In [4]:
response = openai.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)

response

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

Typically, what we're most interested in is the `choices[0].message.content`:

In [ ]:
response.choices[0].message.content

To make things a bit easier to modify parameters, we can generalize the calls as a function, including parameters and messages.

In [ ]:
def get_completion(params, messages):
    """ GET completion from openai api"""

    response = openai.chat.completions.create(
        model = params['model'],
        messages = messages,
        temperature = params['temperature'],
        max_tokens = params['max_tokens'],
        top_p = params['top_p'],
        frequency_penalty = params['frequency_penalty'],
        presence_penalty = params['presence_penalty'],
    )
    return response

There's a variety of different parameters that are common with LLM's. Since LLM's are really just word predictors (auto-complete, distributions over vocabulary), they require [different sampling methods](https://huyenchip.com/2024/01/16/sampling.html) to get the next word.

| Parameter          | Description                                                                                                                                                   |
|--------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `model`            | Specifies the model to be used for generating responses. Different models may have different capabilities, size, and performance characteristics.             |

Here's [an outline](https://www.promptingguide.ai/introduction/settings) of different common LLM parameters.


In [ ]:
!openai --help

In [ ]:
def set_open_params(
    model="gpt-3.5-turbo",
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
):
    """ set openai parameters"""

    openai_params = {}

    openai_params['model'] = model
    openai_params['temperature'] = temperature
    openai_params['max_tokens'] = max_tokens
    openai_params['top_p'] = top_p
    openai_params['frequency_penalty'] = frequency_penalty
    openai_params['presence_penalty'] = presence_penalty
    return openai_params

In [ ]:
# basic example

params = set_open_params(temperature = 0.7)

prompt = "Write a haiku about a beagle."

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)

In [ ]:
IPython.display.display(IPython.display.Markdown(response.choices[0].message.content))

### Temperature


| Parameter          | Description                                                                                                                                                   |
|--------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `temperature`      | Controls randomness in the response generation. A higher temperature results in more random responses, while a lower temperature produces more deterministic responses. |

Try to modify the temperature from 0 to 1.

> In terms of application, you might want to use a lower temperature value for tasks like fact-based QA to encourage more factual and concise responses. For poem generation or other creative tasks, it might be beneficial to increase the temperature value.

In [ ]:
import re

def split_on_capital(text):
    return re.findall(r'[A-Z][^A-Z]*', text)

# Experiment with different temperature values
for temp in [0, 0.2, 0.4, 0.6, 0.8, 1]:
    params = set_open_params(temperature=temp)
    response = get_completion(params, messages)
    lines = split_on_capital(response.choices[0].message.content.strip())
    print(f"Response with temperature={temp}:\n")
    for line in lines:
        IPython.display.display(IPython.display.Markdown(line))
    print("-------------------\n")

[View how I used ChatGPT to iterate on this code](https://chat.openai.com/share/6211107a-d868-491b-8a74-ea4debb7a760)

> ### 🗒 Info: Is it possible to make LLM's deterministic and reproducibile?

> It's possible by setting a seed and setting temperature equal to 0. But as mentioned in [OpenAI's docs](https://cookbook.openai.com/examples/reproducible_outputs_with_the_seed_parameter), "it's important to note that while the seed ensures consistency, it does not guarantee the quality of the output."

### Top P

Used in nucleus sampling, it defines the probability mass to consider for token generation. A smaller `top_p` leads to more focused sampling.

* `top_p` computes the cumulative probability distribution, and cut off as soon as that distribution exceeds the value of `top_p`. **For example, a `top_p` of 0.3 means that only the tokens comprising the top 30% probability mass are considered.**

* `top_p` shrinks or grows the "pool" of available tokens to choose from, the domain to select over. 1=big pool, 0=small pool. Within that pool, each token has a probability of coming next.

Added in a `jinja2` template too (this is optional, but commonly used).

In [ ]:
%%capture
!pip install textstat

In [ ]:
import re
import json
from textstat import flesch_kincaid_grade
from jinja2 import Template

# Define the prompt using a Jinja template
template = Template("""
{
    "role": "user",
    "content": "{{ prompt }}"
}
""")
prompt = "Generate a unique and creative story idea involving time travel."

# Function to count unique words
def count_unique_words(text):
    words = re.findall(r'\b\w+\b', text.lower())
    return len(set(words))

# Generate the message using the template
messages_str = template.render(prompt=prompt)
messages = [json.loads(messages_str)]

for top_p in [0.1, 0.5, 0.95]:
    params = set_open_params(top_p=top_p)
    response = get_completion(params, messages)
    text = response.choices[0].message.content
    unique_word_count = count_unique_words(text)
    fk_score = flesch_kincaid_grade(text)
    print(f"Response with top_p={top_p}:\n")
    print(f"Unique word count: {unique_word_count}")
    print(f"Flesch-Kincaid Grade Level: {fk_score}")
    IPython.display.display(IPython.display.Markdown(text))
    print("-------------------\n")


> **If you are looking for exact and factual answers keep this low. If you are looking for more diverse responses, increase to a higher value.** If you use Top P it means that only the tokens comprising the top_p probability mass are considered for responses, so a low top_p value selects the most confident responses. This means that a high top_p value will enable the model to look at more possible words, including less likely ones, leading to more diverse outputs. **The general recommendation is to alter temperature or Top P but not both.**

In [ ]:
prompt = "What day is it?"

# Generate the message using the template and convert it to a dictionary
messages_str = template.render(prompt=prompt)
messages = [json.loads(messages_str)]

for top_p in [0.5, 0.8, 0.95]:
    params = set_open_params(top_p=top_p)
    response = get_completion(params, messages)
    print(f"Response with top_p={top_p}:\n")
    IPython.display.display(IPython.display.Markdown(response.choices[0].message.content))
    print("-------------------\n")

### Max Tokens

Defines the maximum length of the generated response measured in tokens (words or pieces of words). It helps in controlling the verbosity of the response.

* **Prompt**: "Explain how photosynthesis works in plants."
* **Task**: Run the API three times with `max_tokens` set to 50, 100, and 200, respectively.

In [ ]:
# Experiment with different max_tokens values
prompt = "Explain how photosynthesis works in plants."

messages = [
    {
        "role": "user",
        "content": prompt
    }
]


for max_tokens in [50, 100, 200]:
    params = set_open_params(max_tokens=max_tokens)
    response = get_completion(params, messages)
    print(f"Response with max_tokens={max_tokens}:\n")
    IPython.display.display(IPython.display.Markdown(response.choices[0].message.content))
    print("-------------------\n")

### 1.1 Text Summarization

In [ ]:
params = set_open_params(temperature=0.7)

prompt = """Explain the below in one sentence: Antibiotics are a type of medication \
          used to treat bacterial infections. They work by either killing the bacteria \
          or preventing them from reproducing, allowing the body's immune system to \
          fight off the infection. Antibiotics are usually taken orally in the form \
          of pills, capsules, or liquid solutions, or sometimes administered intravenously. \
          They are not effective against viral infections, and using them inappropriately \
          can lead to antibiotic resistance."""

# Generate the message using the template and convert it to a dictionary
messages_str = template.render(prompt=prompt)
messages = [json.loads(messages_str)]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

> **Exercise**: Instruct the model to explain the paragraph in one sentence like "I am 5". Do you see any differences?

### 1.2 Question Answering

In [ ]:
prompt = """Answer the question based on the context below. \
 Keep the answer short and concise. \
 Respond 'Unsure about answer' if not sure about the answer. \
 Context: Teplizumab traces its roots to a New Jersey drug company called Ortho Pharmaceutical. \
 There, scientists generated an early version of the antibody, dubbed OKT3. \
 Originally sourced from mice, the molecule was able to bind to the surface of T cells \
 and limit their cell-killing potential. In 1986, it was approved to help prevent organ \
 rejection after kidney transplants, making it the first therapeutic antibody allowed \
 for human use. \
 Question: What was OKT3 originally sourced from? \
 Answer:"""

# Generate the message using the template and convert it to a dictionary
messages_str = template.render(prompt=prompt)
messages = [json.loads(messages_str)]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)


Context obtained from here: https://www.nature.com/articles/d41586-023-00400-x

> **Exercise**: Edit prompt and get the model to respond that it isn't sure about the answer.

### 1.3 Text Classification

In [ ]:
prompt = """Classify the text into neutral, negative or positive. \
                                                  \
            Text: I think the food was amazing!   \
                                                  \
            Sentiment:"""

# Generate the message using the template and convert it to a dictionary
messages_str = template.render(prompt=prompt)
messages = [json.loads(messages_str)]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

> **Exercise**: Modify the prompt to instruct the model to provide an explanation to the answer selected.

### 1.4 Role Playing

In [ ]:
prompt = """The following is a conversation with an AI research assistant. \
The assistant tone is technical and scientific. \
                                                \
Human: Hello, who are you? \
AI: Greeting! I am an AI research assistant. How can I help you today? \
Human: Can you tell me about the creation of blackholes? \
AI:"""

# Generate the message using the template and convert it to a dictionary
messages_str = template.render(prompt=prompt)
messages = [json.loads(messages_str)]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

> **Exercise**: Modify the prompt to instruct the model to keep AI responses concise and short.

### 1.5 Code Generation

In [ ]:
prompt = """Develop a small Python API that functions as a virtual plant care assistant."""

# Generate the message using the template and convert it to a dictionary
messages_str = template.render(prompt=prompt)
messages = [json.loads(messages_str)]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

### 1.6 Reasoning

In [ ]:
prompt = """The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. \
                                                                                              \
Solve by breaking the problem into steps.                                                     \
                                                                                              \
First, identify the odd numbers, add them, and indicate whether the result is odd or even."""

# Generate the message using the template and convert it to a dictionary
messages_str = template.render(prompt=prompt)
messages = [json.loads(messages_str)]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

> **Exercise**: Improve the prompt to have a better structure and output format.

### 1.7 Brain storming

In [ ]:
prompt = """Generate 10 different names for a grunge rock bands based on dog names. Keep it to only 2 word band names."""

# Generate the message using the template and convert it to a dictionary
messages_str = template.render(prompt=prompt)
messages = [json.loads(messages_str)]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

### 1.8 JSON generation

Another new feature in OpenAI is [JSON mode](https://platform.openai.com/docs/guides/text-generation/json-mode).



In [ ]:
response = openai.chat.completions.create(
  model="gpt-3.5-turbo",
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
    {"role": "user", "content": "Generate 5 doctor's notes as a 'text' field that precribes drugs, dosages, form, duration, and frequency. Then in JSON have an 'entities' nested dictionary with each of the entities"}
  ]
)
print(response.choices[0].message.content)

> ### 🗒 Info
> - When using JSON mode, always instruct the model to produce JSON via some message in the conversation, for example via your system message. If you don't include an explicit instruction to generate JSON, the model may generate an unending stream of whitespace and the request may run continually until it reaches the token limit. To help ensure you don't forget, the API will throw an error if the string "JSON" does not appear somewhere in the context.

> - The JSON in the message the model returns may be partial (i.e. cut off) if finish_reason is length, which indicates the generation exceeded max_tokens or the conversation exceeded the token limit. To guard against this, check finish_reason before parsing the response.

> - JSON mode will not guarantee the output matches any specific schema, only that it is valid and parses without errors.

## 2. Advanced Prompting Techniques

Objectives:

- Cover more advanced techniques for prompting: few-shot, chain-of-thoughts,...

### 2.2 Few-shot prompts

[Default v2 Prompt from Prodigy](https://prodi.gy/docs/large-language-models#more-config).

In [ ]:
prompt = """
You are an expert Named Entity Recognition (NER) system. Your task is to accept Text as input and extract named entities for the set of predefined entity labels.

From the Text input provided, extract named entities for each label in the following format:

DISH: <comma delimited list of strings>
INGREDIENT: <comma delimited list of strings>
EQUIPMENT: <comma delimited list of strings>

Below are definitions of each label to help aid you in what kinds of named entities to extract for each label.
Assume these definitions are written by an expert and follow them closely.

DISH: Extract the name of a known dish.
INGREDIENT: Extract the name of a cooking ingredient, including herbs and spices.
EQUIPMENT: Extract any mention of cooking equipment. e.g. oven, cooking pot, grill

Below are some examples (only use these as a guide):

Text:
'''
You can't get a great chocolate flavor with carob.
'''

INGREDIENT: carob

Text:
'''
You can probably sand-blast it if it's an anodized aluminum pan.
'''

INGREDIENT:
EQUIPMENT: anodized aluminum pan


Here is the text that needs labeling:

Text:
'''
In Silicon Valley, a Voice of Caution Guides a High-Flying Uber
'''
"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

In [ ]:
prompt = """The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

In [ ]:
prompt = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The answer is False.

The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

### 2.3 Chain-of-Thought (CoT) Prompting

In [ ]:
prompt = """I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?

Let's think step by step."""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

### 2.5 Self-Consistency
As an **optional** exercise, check examples in our [guide](https://github.com/dair-ai/Prompt-Engineering-Guide/blob/main/guides/prompts-advanced-usage.md#self-consistency) and try them here.

### 2.6 Generate Knowledge Prompting

As an **optional** exercise, check examples in our [guide](https://github.com/dair-ai/Prompt-Engineering-Guide/blob/main/guides/prompts-advanced-usage.md#generated-knowledge-prompting) and try them here.

### 3. LangChain

This is adopted from [this notebook](https://github.com/dair-ai/Prompt-Engineering-Guide/blob/main/notebooks/pe-chatgpt-langchain.ipynb).

In [5]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [6]:
# chat mode instance
chat = ChatOpenAI(temperature=0)

In [7]:
USER_INPUT = "I love programming."
FINAL_PROMPT = """Classify the text into neutral, negative or positive.

Text: {user_input}.
Sentiment:"""

chat.invoke([HumanMessage(content=FINAL_PROMPT.format(user_input=USER_INPUT))])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

---